# Create Psuedo Synthetic GNSS Velocity Waveforms
Notebook to explain workflow

Steps
1. previously processed ambient 5hz GNSS TDCP dataset in [Dittmann, et al. 2022](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2022JB024854)
2. convert 'ambient' timeseries (txt files) into [sac format](http://www.adc1.iris.edu/files/sac-manual/manual/file_format.html) (intermediate step because at this time I didn't want to fully refactor obspy inputs) 
3. read all sac files into horizontal (E/N) and vertical (V) PPSD objects.  This is obspy code refactored to support our use case of stacking different stations and two components
4. Use PPSD to generate synthetic noise timeseries to add to [NGAWest2](https://ngawest2.berkeley.edu/) Strong Motion waveforms.

### Step 2 

convert 'ambient' timeseries (txt files) into [sac format](http://www.adc1.iris.edu/files/sac-manual/manual/file_format.html) (intermediate step because at this time didn't want to fully refactor obspy inputs)

In [ ]:
%run ../bin/noise/ambient2sac.py

### Step 3. 
read all sac files into horizontal (E/N) and vertical (V) PPSD objects.  This is obspy code refactored to support our use case of stacking different stations and two components (in the horizontal case) into a single PPSD object.

In [ ]:
%run ../bin/noise/sac2noise.py

### Step 4. Generate Timeseries

#### Signal Waveforms
* Transferred waveforms are from NGAWest2 Strong Motion catalog.  
* We use signals from events M4.8-M8.2, <70km Rrup
* We use velocity waveforms, downsampled to 5Hz using lowpass with cutoff freq of 2.5Hz (run forward and backwards)

#### Noise Timeseries
* We create 7 unique timeseries (3 component) from 7 discrete noise levels [5,20,35,50,65,80,95]
* Each has a random initial buffer added (between 0 and 300 samples) to offset onset of motion
* Each slice of PPSD spectrum is applied to a random guassian timeseries to create unique stochastic noise timeseries.

#### Pseudo Synthetic timeseries
* vel_ts object is sum of signal waveform + noise timeseries
* labels are assigned via an STA/LTA applied to the 'zero noise' ngaw2 waveform
* metadata attached to parquet object includes 
```python
meta_content = {
            "station":meta_array[3],
            "year":meta_array[2],
            "eq_name":meta_array[1],
            "record_number":meta_array[0],
            "magnitude":meta_array[4],
            "mechanism":meta_array[5],
            "Rjb":meta_array[6],
            "Rrup":meta_array[7],
            "Vs30": meta_array[8],
            "noise_lev": str(noise_lev),
        }
```

Timeseries are stored as parquet files with naming {record_number}_{noise_level}.pq




In [ ]:
%run ../bin/generate_ts/generate_ts_mp.py